<a href="https://colab.research.google.com/github/vbanai/Multi-label-classification_DOLPHIN_HORSE_Part1_Pytorch-Conv2DandResnet9_Tensorflow_DNNandCNN/blob/main/Pytorch_Resnet9_Transfer_L_Horse_Dolphin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -------------   HORSE/DOLPHIN EXECRCISE WITH RESNET 9 ARCHITECTURE (MULTILABEL) +Data Augmentation----------------

import os
import torch
import torchvision
import tarfile
from torch.utils.data import Dataset
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from google.colab import files
import matplotlib.pyplot as plt
from torch.utils.data.dataloader import DataLoader
import cv2
from torchvision.datasets import MNIST
import torch.nn.functional as F
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import torch.nn as nn
from PIL import Image
from torchvision.transforms import ToTensor
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import pandas as pd
from sklearn.model_selection import train_test_split



In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!ls "/content/valid"

In [ ]:
#delete folder with files
!rm -rf "/content/test"

In [ ]:
path="."
os.chdir(path)
os.makedirs("train")
os.makedirs("valid")
os.makedirs("test")

path="./train"
os.chdir(path)
os.makedirs("horse")
os.makedirs("dolphin")
os.makedirs("dolphin_horse")

path="/content/valid"
os.chdir(path)
os.makedirs("horse")
os.makedirs("dolphin")
os.makedirs("dolphin_horse")

path="/content/test"
os.chdir(path)
os.makedirs("horse")
os.makedirs("dolphin")
os.makedirs("dolphin_horse")

!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('/content/drive/MyDrive/dolphin_train.rar').extractall('/content/train/dolphin/')
Archive('/content/drive/MyDrive/horse_train.rar').extractall('/content/train/horse/')
Archive("/content/drive/MyDrive/dolphin_horse_train.rar").extractall('/content/train/dolphin_horse/')
Archive('/content/drive/MyDrive/horse_valid.rar').extractall('/content/valid/horse/')
Archive('/content/drive/MyDrive/dolphin_valid.rar').extractall('/content/valid/dolphin/')
Archive('/content/drive/MyDrive/dolphin_horse_valid.rar').extractall('/content/valid/dolphin_horse/')
Archive('/content/drive/MyDrive/DolphinTest.rar').extractall('/content/test/dolphin/')
Archive('/content/drive/MyDrive/HorseTest.rar').extractall('/content/test/horse/')
Archive("/content/drive/MyDrive/DolphinHorseTest.rar").extractall('/content/test/dolphin_horse/')

     |████████████████████████████████| 81kB 5.7MB/s 


In [ ]:
#------------------ STAT FOR DOLPHIN TRAIN------
transform = transforms.Compose([
        transforms.ToTensor() ])

datadir="/content/train/dolphin/dolphin_train" 


#datadir="/content/train/" 
#train_categories=['horse/horse_train/', 'dolphin/dolphin_train/', 'dolphin_horse/dolphin_horse_train/']

dataset_stat=[]
def create_stat_data():
 # for category in train_categories:
  #  path_train=os.path.join(datadir, category)
  for img in os.listdir(datadir):  
    try:
      img_array=cv2.imread(os.path.join(datadir, img))#, cv2.IMREAD_GRAYSCALE)
      new_array=cv2.resize(img_array, (128, 128))  
      new_array1=new_array/255 
      new_array2=transform(new_array1)
      dataset_stat.append(new_array2)
    except Exception as e:
      print(e)

create_stat_data()
arr_stack_stack=torch.stack(dataset_stat)


stat_train_loader=DataLoader(arr_stack_stack, batch_size=32, shuffle=False)

def get_mean_and_std(loader):
  mean=0.
  std=0.
  total_images_count=0
  for images in loader:
    image_count_in_a_bach=images.size(0)
    images=images.view(image_count_in_a_bach, images.size(1), -1)
    mean +=images.mean(2).sum(0)
    std +=images.std(2).sum(0)
    total_images_count +=image_count_in_a_bach

  mean /=total_images_count
  std /=total_images_count

  return mean, std

get_mean_and_std(stat_train_loader)


OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



(tensor([0.6159, 0.5451, 0.4075], dtype=torch.float64),
 tensor([0.1874, 0.1824, 0.1982], dtype=torch.float64))

In [ ]:

##########  -------------------DOLPHIN SET TRAIN----------------------

from PIL import Image



stats = ((0.6159, 0.5451, 0.4075), (0.1874, 0.1824, 0.1982))
train_tfms2 = transforms.Compose([
                         transforms.ToPILImage(),
                         transforms.Resize((128, 128)),
                         transforms.RandomHorizontalFlip(),
                         transforms.RandomCrop(128, padding=4, padding_mode='reflect'),
                         transforms.ToTensor(),
                         transforms.Normalize(*stats,inplace=True)]) 
                   
datadir="/content/train/" 
train_categories=['horse/horse_train/', 'dolphin/dolphin_train/', 'dolphin_horse/dolphin_horse_train/']

dataset_train=[]
def create_train_data():
  for category in train_categories:
    path_train=os.path.join(datadir, category)
    #if category=="horse/horse_train/":
     # class_num_train=torch.tensor([1,0], dtype=torch.float32)
    if category=="dolphin/dolphin_train/":
      class_num_train=torch.tensor([0,1], dtype=torch.float32)
    #if category=="dolphin_horse/dolphin_horse_train/":
    #  class_num_train=torch.tensor([1,1], dtype=torch.float32)
      for img in os.listdir(path_train):  
        try:   
          img_array=cv2.imread(os.path.join(path_train, img))#, cv2.IMREAD_GRAYSCALE)
          #img_array2=np.array(img_array, dtype=np.float32)
          #new_array=cv2.resize(img_array, (128, 128))  
          #new_array0=img_array/255 
          new_array1=train_tfms2(img_array)
          #new_array2=new_array1/255
          #new_array3=torch.tensor(new_array2, dtype=torch.float32)
          dataset_train.append([new_array1, class_num_train])
        except Exception as e:
          print(e)

create_train_data()



pic should be Tensor or ndarray. Got <class 'NoneType'>.


In [ ]:
#------------------ Horse STAT TRAIN-------
transform = transforms.Compose([
        transforms.ToTensor() ])

datadir="/content/train/horse/horse_train" 


dataset_stat=[]
def create_stat_data():
 
    for img in os.listdir(datadir):  
      try:
        img_array=cv2.imread(os.path.join(datadir, img))#, cv2.IMREAD_GRAYSCALE)
        new_array=cv2.resize(img_array, (128, 128))  
        new_array1=new_array/255 
        new_array2=transform(new_array1)
        dataset_stat.append(new_array2)
      except Exception as e:
        print(e)

create_stat_data()
arr_stack_stack=torch.stack(dataset_stat)

horse_train_loader=DataLoader(arr_stack_stack, batch_size=32, shuffle=False)

def get_mean_and_std(loader):
  mean=0.
  std=0.
  total_images_count=0
  for images in loader:
    image_count_in_a_bach=images.size(0)
    images=images.view(image_count_in_a_bach, images.size(1), -1)
    mean +=images.mean(2).sum(0)
    std +=images.std(2).sum(0)
    total_images_count +=image_count_in_a_bach

  mean /=total_images_count
  std /=total_images_count

  return mean, std

get_mean_and_std(horse_train_loader)

OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



(tensor([0.4061, 0.4831, 0.5015], dtype=torch.float64),
 tensor([0.2390, 0.2349, 0.2310], dtype=torch.float64))

In [ ]:
##########  -------------------HORSE SET TRAIN----------------------

#from PIL import Image



stats = ((0.4061, 0.4831, 0.5015), (0.2390, 0.2349, 0.2310))

train_tfms2 = transforms.Compose([
                         transforms.ToPILImage(),
                         transforms.Resize((128, 128)),
                         transforms.RandomHorizontalFlip(),
                         transforms.RandomCrop(128, padding=4, padding_mode='reflect'),
                         transforms.ToTensor(),
                         transforms.Normalize(*stats,inplace=True)]) 
                   
datadir="/content/train/" 
train_categories=['horse/horse_train/', 'dolphin/dolphin_train/', 'dolphin_horse/dolphin_horse_train/']


def create_train_data():
  for category in train_categories:
    path_train=os.path.join(datadir, category)
    if category=="horse/horse_train/":
      class_num_train=torch.tensor([1,0], dtype=torch.float32)
    #if category=="dolphin/dolphin_train/":
    #  class_num_train=torch.tensor([0,1], dtype=torch.float32)
    #if category=="dolphin_horse/dolphin_horse_train/":
     # class_num_train=torch.tensor([1,1], dtype=torch.float32)
      for img in os.listdir(path_train):  
        try:   
          img_array=cv2.imread(os.path.join(path_train, img))#, cv2.IMREAD_GRAYSCALE)
          #img_array2=np.array(img_array, dtype=np.float32)
          #new_array=cv2.resize(img_array, (128, 128))  
          #new_array0=img_array/255 
          new_array1=train_tfms2(img_array)
          #new_array2=new_array1/255
          #new_array3=torch.tensor(new_array2, dtype=torch.float32)
          dataset_train.append([new_array1, class_num_train])
        except Exception as e:
          print(e)

create_train_data()


pic should be Tensor or ndarray. Got <class 'NoneType'>.
pic should be Tensor or ndarray. Got <class 'NoneType'>.


In [ ]:
#-------------- STAT FOR HORSE&DOLPHIN TRAIN------

transform = transforms.Compose([
        transforms.ToTensor() ])

datadir="/content/train/dolphin_horse/dolphin_horse_train" 


dataset_stat=[]
def create_stat_data():
 
    for img in os.listdir(datadir):  
      try:
        img_array=cv2.imread(os.path.join(datadir, img))#, cv2.IMREAD_GRAYSCALE)
        new_array=cv2.resize(img_array, (128, 128))  
        new_array1=new_array/255 
        new_array2=transform(new_array1)
        dataset_stat.append(new_array2)
      except Exception as e:
        print(e)

create_stat_data()
arr_stack_stack=torch.stack(dataset_stat)

horse_train_loader=DataLoader(arr_stack_stack, batch_size=32, shuffle=False)

def get_mean_and_std(loader):
  mean=0.
  std=0.
  total_images_count=0
  for images in loader:
    image_count_in_a_bach=images.size(0)
    images=images.view(image_count_in_a_bach, images.size(1), -1)
    mean +=images.mean(2).sum(0)
    std +=images.std(2).sum(0)
    total_images_count +=image_count_in_a_bach

  mean /=total_images_count
  std /=total_images_count

  return mean, std

get_mean_and_std(horse_train_loader)


In [ ]:
###########------- HORSE&DOLPHIN TRAIN DATA-------


stats = ((0.6082, 0.5902, 0.5349), (0.2793, 0.2695, 0.3072))
train_tfms2 = transforms.Compose([
                         transforms.ToPILImage(),
                         transforms.Resize((128, 128)),
                         transforms.RandomHorizontalFlip(),
                         transforms.RandomCrop(128, padding=4, padding_mode='reflect'),
                         transforms.ToTensor(),
                         transforms.Normalize(*stats,inplace=True)]) 
                   
datadir="/content/train/" 
train_categories=['horse/horse_train/', 'dolphin/dolphin_train/', 'dolphin_horse/dolphin_horse_train/']


def create_train_data():
  for category in train_categories:
    path_train=os.path.join(datadir, category)
    #if category=="horse/horse_train/":
     # class_num_train=torch.tensor([1,0], dtype=torch.float32)
    #if category=="dolphin/dolphin_train/":
    #  class_num_train=torch.tensor([0,1], dtype=torch.float32)
    if category=="dolphin_horse/dolphin_horse_train/":
      class_num_train=torch.tensor([1,1], dtype=torch.float32)
      for img in os.listdir(path_train):  
        try:   
          img_array=cv2.imread(os.path.join(path_train, img))#, cv2.IMREAD_GRAYSCALE)
          #img_array2=np.array(img_array, dtype=np.float32)
          #new_array=cv2.resize(img_array, (128, 128))  
          #new_array0=img_array/255 
          new_array1=train_tfms2(img_array)
          #new_array2=new_array1/255
          #new_array3=torch.tensor(new_array2, dtype=torch.float32)
          dataset_train.append([new_array1, class_num_train])
        except Exception as e:
          print(e)

create_train_data()


In [ ]:
#########------Final TRAIN DATASET CREATION------

import random
random.shuffle(dataset_train)

inputs_train=[]
targets_train0=[]

for image, label in dataset_train:
  inputs_train.append(image)
  targets_train0.append(label)

#---creating np array from the input images
#arr_inputs_train = np.array(inputs_train)
#arr_inputs_float_train=arr_inputs_train.astype('float32')


#---convert np array of the input images to tensors-----
#arr_tensor_train = [torch.from_numpy(item).float() for item in arr_inputs_float_train]
arr_stack_train=torch.stack(inputs_train)

#--- reshape the input images tensor
arr_stack_train_reshape=arr_stack_train.reshape(-1,3,128,128) 
targets_train=torch.stack(targets_train0)
#targets_torch_train=torch.tensor(targets_train, dtype=torch.float32)

---------------TRAIN FINAL END----------------

In [ ]:
plt.imshow(train_final[2][0].permute(1,2,0))

-------VALIDATION DATASET------

In [ ]:


#------------------ STAT FOR VALID DOLPHIN-------
transform = transforms.Compose([
        transforms.ToTensor() ])

datadir="/content/valid/dolphin/dolphin_valid" 


dataset_stat=[]
def create_stat_data():
 
    for img in os.listdir(datadir):  
      try:
        img_array=cv2.imread(os.path.join(datadir, img))#, cv2.IMREAD_GRAYSCALE)
        new_array=cv2.resize(img_array, (128, 128))  
        new_array1=new_array/255 
        new_array2=transform(new_array1)
        dataset_stat.append(new_array2)
      except Exception as e:
        print(e)

create_stat_data()
arr_stack_stack=torch.stack(dataset_stat)

horse_train_loader=DataLoader(arr_stack_stack, batch_size=32, shuffle=False)

def get_mean_and_std(loader):
  mean=0.
  std=0.
  total_images_count=0
  for images in loader:
    image_count_in_a_bach=images.size(0)
    images=images.view(image_count_in_a_bach, images.size(1), -1)
    mean +=images.mean(2).sum(0)
    std +=images.std(2).sum(0)
    total_images_count +=image_count_in_a_bach

  mean /=total_images_count
  std /=total_images_count

  return mean, std

get_mean_and_std(horse_train_loader)

(tensor([0.6489, 0.5334, 0.3283], dtype=torch.float64),
 tensor([0.1639, 0.1617, 0.1790], dtype=torch.float64))

In [ ]:
###############----- Valid set creation - Dolphin-------

stats = ((0.6489, 0.5334, 0.3283), (0.1639, 0.1617, 0.1790))

train_tfms2 = transforms.Compose([
                         transforms.ToPILImage(),
                         transforms.Resize((128, 128)),
                         #transforms.RandomHorizontalFlip(),
                         #transforms.RandomCrop(128, padding=4, padding_mode='reflect'),
                         transforms.ToTensor(),
                         transforms.Normalize(*stats,inplace=True)]) 
                   
datadir="/content/valid/" 
train_categories=['horse/horse_valid/', 'dolphin/dolphin_valid/', 'dolphin_horse/dolphin_horse_valid/']

dataset_valid=[]
def create_valid_data():
  for category in train_categories:
    path_train=os.path.join(datadir, category)
    #if category=="horse/horse_valid/":
     # class_num_valid=torch.tensor([1,0], dtype=torch.float32)
    if category=="dolphin/dolphin_valid/":
      class_num_valid=torch.tensor([0,1], dtype=torch.float32)
    #if category=="dolphin_horse/dolphin_horse_valid/":
    #  class_num_valid=torch.tensor([1,1], dtype=torch.float32)
      for img in os.listdir(path_train):  
        try:   
          img_array=cv2.imread(os.path.join(path_train, img))#, cv2.IMREAD_GRAYSCALE)
          #img_array2=np.array(img_array, dtype=np.float32)
          #new_array=cv2.resize(img_array, (128, 128))  
          #new_array0=img_array/255 
          new_array1=train_tfms2(img_array)
          #new_array2=new_array1/255
          #new_array3=torch.tensor(new_array2, dtype=torch.float32)
          dataset_valid.append([new_array1, class_num_valid])
        except Exception as e:
          print(e)

create_valid_data()

In [ ]:
#-------------STAT FOR HORSE VALIDATION DATASET---------

transform = transforms.Compose([
        transforms.ToTensor() ])

datadir="/content/valid/horse/horse_valid" 


dataset_stat=[]
def create_stat_data():
 
    for img in os.listdir(datadir):  
      try:
        img_array=cv2.imread(os.path.join(datadir, img))#, cv2.IMREAD_GRAYSCALE)
        new_array=cv2.resize(img_array, (128, 128))  
        new_array1=new_array/255 
        new_array2=transform(new_array1)
        dataset_stat.append(new_array2)
      except Exception as e:
        print(e)

create_stat_data()
arr_stack_stack=torch.stack(dataset_stat)

horse_train_loader=DataLoader(arr_stack_stack, batch_size=32, shuffle=False)

def get_mean_and_std(loader):
  mean=0.
  std=0.
  total_images_count=0
  for images in loader:
    image_count_in_a_bach=images.size(0)
    images=images.view(image_count_in_a_bach, images.size(1), -1)
    mean +=images.mean(2).sum(0)
    std +=images.std(2).sum(0)
    total_images_count +=image_count_in_a_bach

  mean /=total_images_count
  std /=total_images_count

  return mean, std

get_mean_and_std(horse_train_loader)

In [ ]:
############### -----------Horse_validation set creation----------

stats = ((0.3924, 0.5006, 0.5159), (0.2448, 0.2328, 0.2217))

train_tfms2 = transforms.Compose([
                         transforms.ToPILImage(),
                         transforms.Resize((128, 128)),
                         #transforms.RandomHorizontalFlip(),
                         #transforms.RandomCrop(128, padding=4, padding_mode='reflect'),
                         transforms.ToTensor(),
                         transforms.Normalize(*stats,inplace=True)]) 
                   
datadir="/content/valid/" 
train_categories=['horse/horse_valid/', 'dolphin/dolphin_valid/', 'dolphin_horse/dolphin_horse_valid/']


def create_valid_data():
  for category in train_categories:
    path_train=os.path.join(datadir, category)
    if category=="horse/horse_valid/":
      class_num_valid=torch.tensor([1,0], dtype=torch.float32)
    #if category=="dolphin/dolphin_valid/":
     # class_num_valid=torch.tensor([0,1], dtype=torch.float32)
    #if category=="dolphin_horse/dolphin_horse_valid/":
    #  class_num_valid=torch.tensor([1,1], dtype=torch.float32)
      for img in os.listdir(path_train):  
        try:   
          img_array=cv2.imread(os.path.join(path_train, img))#, cv2.IMREAD_GRAYSCALE)
          #img_array2=np.array(img_array, dtype=np.float32)
          #new_array=cv2.resize(img_array, (128, 128))  
          #new_array0=img_array/255 
          new_array1=train_tfms2(img_array)
          #new_array2=new_array1/255
          #new_array3=torch.tensor(new_array2, dtype=torch.float32)
          dataset_valid.append([new_array1, class_num_valid])
        except Exception as e:
          print(e)

create_valid_data()

In [ ]:
#-------------STAT FOR HORSED/DOLPIN VALIDATION DATASET----------
transform = transforms.Compose([
        transforms.ToTensor() ])

datadir="/content/valid/dolphin_horse/dolphin_horse_valid" 


dataset_stat=[]
def create_stat_data():
 
    for img in os.listdir(datadir):  
      try:
        img_array=cv2.imread(os.path.join(datadir, img))#, cv2.IMREAD_GRAYSCALE)
        new_array=cv2.resize(img_array, (128, 128))  
        new_array1=new_array/255 
        new_array2=transform(new_array1)
        dataset_stat.append(new_array2)
      except Exception as e:
        print(e)

create_stat_data()
arr_stack_stack=torch.stack(dataset_stat)

horse_train_loader=DataLoader(arr_stack_stack, batch_size=32, shuffle=False)

def get_mean_and_std(loader):
  mean=0.
  std=0.
  total_images_count=0
  for images in loader:
    image_count_in_a_bach=images.size(0)
    images=images.view(image_count_in_a_bach, images.size(1), -1)
    mean +=images.mean(2).sum(0)
    std +=images.std(2).sum(0)
    total_images_count +=image_count_in_a_bach

  mean /=total_images_count
  std /=total_images_count

  return mean, std

get_mean_and_std(horse_train_loader)

(tensor([0.6164, 0.5990, 0.5362], dtype=torch.float64),
 tensor([0.3014, 0.2810, 0.3143], dtype=torch.float64))

In [ ]:
#############----------- DolphinHorse validity set creation----------------

stats = ((0.6164, 0.5990, 0.5362), (0.3014, 0.2810, 0.3143))
train_tfms2 = transforms.Compose([
                         transforms.ToPILImage(),
                         transforms.Resize((128, 128)),
                         #transforms.RandomHorizontalFlip(),
                         #transforms.RandomCrop(128, padding=4, padding_mode='reflect'),
                         transforms.ToTensor(),
                         transforms.Normalize(*stats,inplace=True)]) 
                   
datadir="/content/valid/" 
train_categories=['horse/horse_valid/', 'dolphin/dolphin_valid/', 'dolphin_horse/dolphin_horse_valid/']


def create_valid_data():
  for category in train_categories:
    path_train=os.path.join(datadir, category)
    #if category=="horse/horse_valid/":
     # class_num_valid=torch.tensor([1,0], dtype=torch.float32)
    #if category=="dolphin/dolphin_valid/":
     # class_num_valid=torch.tensor([0,1], dtype=torch.float32)
    if category=="dolphin_horse/dolphin_horse_valid/":
      class_num_valid=torch.tensor([1,1], dtype=torch.float32)
      for img in os.listdir(path_train):  
        try:   
          img_array=cv2.imread(os.path.join(path_train, img))#, cv2.IMREAD_GRAYSCALE)
          #img_array2=np.array(img_array, dtype=np.float32)
          #new_array=cv2.resize(img_array, (128, 128))  
          #new_array0=img_array/255 
          new_array1=train_tfms2(img_array)
          #new_array2=new_array1/255
          #new_array3=torch.tensor(new_array2, dtype=torch.float32)
          dataset_valid.append([new_array1, class_num_valid])
        except Exception as e:
          print(e)

create_valid_data()

In [ ]:
#-----VAlIDTY FINAL-------
import random
random.shuffle(dataset_valid)

inputs_valid=[]
targets_valid0=[]

for image, label in dataset_valid:
  inputs_valid.append(image)
  targets_valid0.append(label)

#---creating np array from the input images
#arr_inputs_train = np.array(inputs_train)
#arr_inputs_float_train=arr_inputs_train.astype('float32')


#---convert np array of the input images to tensors-----
#arr_tensor_train = [torch.from_numpy(item).float() for item in arr_inputs_float_train]
arr_stack_valid=torch.stack(inputs_valid)

#--- reshape the input images tensor
arr_stack_valid_reshape=arr_stack_valid.reshape(-1,3,128,128) 
targets_valid=torch.stack(targets_valid0)
#targets_torch_train=torch.tensor(targets_train, dtype=torch.float32)

In [ ]:
batch_size=32

dataset_tensor_train=TensorDataset(arr_stack_train_reshape, targets_train)
dataset_tensor_valid=TensorDataset(arr_stack_valid_reshape, targets_valid)

train_dl = DataLoader(dataset_tensor_train, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(dataset_tensor_valid, batch_size, shuffle=True, num_workers=2, pin_memory=True)




In [ ]:

batch_size=32

dataset=TensorDataset(arr_stack_train_reshape, targets_train)
dataset_tensor_test=TensorDataset(arr_stack_valid_reshape, targets_valid)

dataset_tensor_train, dataset_tensor_valid =train_test_split(dataset, test_size=0.15, random_state=42)

train_dl = DataLoader(dataset_tensor_train, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(dataset_tensor_valid, batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_dl=DataLoader(dataset_tensor_test, batch_size, shuffle=True, num_workers=2, pin_memory=True)


In [ ]:
stats = ((0.4079, 0.5456, 0.6163), (0.1924, 0.1763, 0.1816))

def denormalize(images, means, stds):
    means = torch.tensor(means).reshape(1, 3, 1, 1)
    stds = torch.tensor(stds).reshape(1, 3, 1, 1)
    return images * stds + means

example = iter(train_dl)
sample, label = example.next()

for i in range(9):
  plt.subplot(3,3,i+1)
  denorm_images = denormalize(sample[i], *stats)
  plt.imshow(sample[i].permute(1,2,0))



In [ ]:
def show_batch(train_dl):
    for images, labels in train_dl:
        fig, ax = plt.subplots(figsize=(12, 12))
        #ax.set_xticks([]); ax.set_yticks([])
        denorm_images = denormalize(images, *stats)
        ax.imshow(make_grid(denorm_images[:64], nrow=8).permute(1, 2, 0).clamp(0,1))
        break

show_batch(train_dl)

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
device

device(type='cuda')

In [ ]:
# ~ and &
# ~ = NOT
# & = AND different and /or


# Bitwise NOT and bitwise and operator
# Checkout https://realpython.com/python-bitwise-operators/ for detailed info
threshold = 0.5
output = np.array([[0.7, 0.6, 0.1, 0.7]])
label = np.array([[1, 1, 1, 1]])
# prob and label are List of BOOLEANs (True or False)
prob = output > threshold #
label = label > threshold

print(prob, label)
(prob & label).sum(1)

In [ ]:
def F_score(output, label, threshold=0.5, beta=1):
    prob = output > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)

    # F2 = 2* precision * recall / (precision + recall)

    return F2.mean(0)

In [ ]:
class ImageClassificationBase(nn.Module):
  
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.binary_cross_entropy(out, labels)  # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        out = self(images)# Generate predictions
        counter=0
        for i in range(len(batch)):
          if out[i][0]>=0.5 and labels[i][0]==1 and out[i][1]<0.5 and labels[i][1]==0:
            counter+=1
          if out[i][0]<0.5 and labels[i][0]==0 and out[i][1]>=0.5 and labels[i][1]==1:
            counter+=1
          if out[i][0]>=0.5 and labels[i][0]==1 and out[i][1]>=0.5 and labels[i][1]==1:
            counter+=1
        check=counter/len(batch)
        loss = F.binary_cross_entropy(out, labels)    # Calculate loss
        score = F_score(out, labels)
        return {'val_loss': loss.detach(), 'val_acc': score, 'Check': check}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        batch_check=[x['Check'] for x in outputs]
        epoch_check=sum(batch_check)/len(batch_check)
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item(), 'Check':epoch_check}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}, check: {}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc'], result['Check']))
        


def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

# Check out ResNeXT from torch vision:
# https://learnopencv.com/multi-label-image-classification-with-pytorch-image-tagging/
# NUS-WIDE dataset


class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = conv_block(in_channels, 64, pool=True)  #64 x64
        self.conv2 = conv_block(64, 128, pool=True)          #32 x 32
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))
        
        self.conv3 = conv_block(128, 256, pool=True) # 16 x 16
        self.conv4 = conv_block(256, 512, pool=True) # 8 x 8
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(2, 2),
                                        nn.MaxPool2d(4), 
                                        nn.Flatten(), 
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes),
                                        nn.Sigmoid())
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [ ]:
# if GPU is not available
model = ResNet9(3, 2)

In [ ]:
model = to_device(ResNet9(3, 2), device)
model


In [ ]:
history = []
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    #history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                               steps_per_epoch=len(train_loader))
    #sched = torch.optim.lr_scheduler.StepLR(opt_func, step_size=100, gamma=0.1)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    #return history

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)


In [ ]:
epochs = 40
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam


# LR Scheduler (On Plateau)
# Loss 0.001, 0.001..... 10 rounds --> Lr = multiplier*Lr

#lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_func, patience=10)

# LR Scheduler (STEP)
#lr_scheduler = torch.optim.lr_scheduler.StepLR(opt_func, step_size=100, gamma=0.1)

In [ ]:
# Callbacks: Early Stopping callback
# https://pytorch-lightning.readthedocs.io/en/latest/common/early_stopping.html

In [ ]:
%%time
fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

plot_accuracies(history)

In [ ]:

train_losses = [x.get('train_loss') for x in history]
val_losses = [x['val_loss'] for x in history]
plt.plot(train_losses, '-bx')
plt.plot(val_losses, '-rx')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Training', 'Validation'])
plt.title('Loss vs. No. of epochs');

In [ ]:

lrs = np.concatenate([x['lrs'] for x in history])
plt.plot(lrs)
plt.xlabel('Batch no.')
plt.ylabel('Learning rate')
plt.title('Learning Rate vs. Batch no.')

In [ ]:
torch.save(model.state_dict(), 'pics.modelparameters')
model2=ResNet9(3,2)
model2.load_state_dict(torch.load('pics.modelparameters'))
model2.state_dict

def predict_image(img, model):
    xb = img.unsqueeze(0)
    yb = model(xb)
    #_, preds  = torch.max(yb, dim=1)
    return yb

In [ ]:
#------------------ STAT FOR TEST DOLPHIN------
transform = transforms.Compose([
        transforms.ToTensor() ])

datadir="/content/test/dolphin/DolphinTest" 


dataset_stat=[]
def create_stat_data():
 
    for img in os.listdir(datadir):  
      try:
        img_array=cv2.imread(os.path.join(datadir, img))#, cv2.IMREAD_GRAYSCALE)
        new_array=cv2.resize(img_array, (128, 128))  
        new_array1=new_array/255 
        new_array2=transform(new_array1)
        dataset_stat.append(new_array2)
      except Exception as e:
        print(e)

create_stat_data()
arr_stack_stack=torch.stack(dataset_stat)


stat_train_loader=DataLoader(arr_stack_stack, batch_size=32, shuffle=False)

def get_mean_and_std(loader):
  mean=0.
  std=0.
  total_images_count=0
  for images in loader:
    image_count_in_a_bach=images.size(0)
    images=images.view(image_count_in_a_bach, images.size(1), -1)
    mean +=images.mean(2).sum(0)
    std +=images.std(2).sum(0)
    total_images_count +=image_count_in_a_bach

  mean /=total_images_count
  std /=total_images_count

  return mean, std

get_mean_and_std(stat_train_loader)

(tensor([0.6165, 0.5122, 0.2994], dtype=torch.float64),
 tensor([0.1571, 0.1548, 0.1889], dtype=torch.float64))

In [ ]:
###########----TEST DATASET DOLPHIN CREATION-----

stats = ((0.6165, 0.5122, 0.2994), (0.1571, 0.1548, 0.1889))
test_tfms3 = transforms.Compose([
                         transforms.ToPILImage(),
                         transforms.Resize((128, 128)),
                         #transforms.RandomHorizontalFlip(),
                         #transforms.RandomCrop(128, padding=4, padding_mode='reflect'),
                         transforms.ToTensor(),
                         transforms.Normalize(*stats,inplace=True)]) 


dir_test="/content/test" 
test_categories=['horse/HorseTest/', 'dolphin/DolphinTest/', 'dolphin_horse/DolphinHorseTest/']
test_list=[]

def create_test_data():
  for category in test_categories:
    path_test=os.path.join(dir_test, category)
    #if category=="horse/HorseTest/":
     # class_num_test=torch.tensor([1,0], dtype=torch.float32)
    if category=="dolphin/DolphinTest/":
      class_num_test=torch.tensor([0,1], dtype=torch.float32)
    #if category=="dolphin_horse/DolphinHorseTest/":
     # class_num_test=torch.tensor([1,1], dtype=torch.float32)
      for img in os.listdir(path_test):  
        try:
          img_array=cv2.imread(os.path.join(path_test, img))  
          new_array0=test_tfms3(img_array) 
          #new_array=cv2.resize(img_array, (128, 128))
          #new_array1=new_array0/255 
          #test_np = np.array(new_array1)
          #test_np_float32=test_np.astype('float32')
          #test_tensor0 = torch.from_numpy(test_np_float32)
          #test_tensor=new_array0.reshape(3,128,128)
          test_list.append([new_array0, class_num_test])
        except Exception as e:
          print(e)

create_test_data()

In [ ]:
#------------------ STAT FOR TEST HORSE------
transform = transforms.Compose([
        transforms.ToTensor() ])

datadir="/content/test/horse/HorseTest" 


dataset_stat=[]
def create_stat_data():
 
    for img in os.listdir(datadir):  
      try:
        img_array=cv2.imread(os.path.join(datadir, img))#, cv2.IMREAD_GRAYSCALE)
        new_array=cv2.resize(img_array, (128, 128))  
        new_array1=new_array/255 
        new_array2=transform(new_array1)
        dataset_stat.append(new_array2)
      except Exception as e:
        print(e)

create_stat_data()
arr_stack_stack=torch.stack(dataset_stat)


stat_train_loader=DataLoader(arr_stack_stack, batch_size=32, shuffle=False)

def get_mean_and_std(loader):
  mean=0.
  std=0.
  total_images_count=0
  for images in loader:
    image_count_in_a_bach=images.size(0)
    images=images.view(image_count_in_a_bach, images.size(1), -1)
    mean +=images.mean(2).sum(0)
    std +=images.std(2).sum(0)
    total_images_count +=image_count_in_a_bach

  mean /=total_images_count
  std /=total_images_count

  return mean, std

get_mean_and_std(stat_train_loader)

(tensor([0.4448, 0.5173, 0.5299], dtype=torch.float64),
 tensor([0.2385, 0.2264, 0.2245], dtype=torch.float64))

In [ ]:
################----TEST DATASET HORSE CREATION-----

stats = ((0.4448, 0.5173, 0.5299), (0.2385, 0.2264, 0.2245))
test_tfms3 = transforms.Compose([
                         transforms.ToPILImage(),
                         transforms.Resize((128, 128)),
                         #transforms.RandomHorizontalFlip(),
                         #transforms.RandomCrop(128, padding=4, padding_mode='reflect'),
                         transforms.ToTensor(),
                         transforms.Normalize(*stats,inplace=True)]) 


dir_test="/content/test" 
test_categories=['horse/HorseTest/', 'dolphin/DolphinTest/', 'dolphin_horse/DolphinHorseTest/']


def create_test_data():
  for category in test_categories:
    path_test=os.path.join(dir_test, category)
    if category=="horse/HorseTest/":
      class_num_test=torch.tensor([1,0], dtype=torch.float32)
    #if category=="dolphin/DolphinTest/":
     # class_num_test=torch.tensor([0,1], dtype=torch.float32)
    #if category=="dolphin_horse/DolphinHorseTest/":
     # class_num_test=torch.tensor([1,1], dtype=torch.float32)
      for img in os.listdir(path_test):  
        try:
          img_array=cv2.imread(os.path.join(path_test, img))  
          new_array0=test_tfms3(img_array) 
          #new_array=cv2.resize(img_array, (128, 128))
          #new_array1=new_array0/255 
          #test_np = np.array(new_array1)
          #test_np_float32=test_np.astype('float32')
          #test_tensor0 = torch.from_numpy(test_np_float32)
          #test_tensor=new_array0.reshape(3,128,128)
          test_list.append([new_array0, class_num_test])
        except Exception as e:
          print(e)

create_test_data()

In [ ]:
#------------------ STAT FOR TEST HORSEDOLPHIN------
transform = transforms.Compose([
        transforms.ToTensor() ])

datadir="/content/test/dolphin_horse/DolphinHorseTest" 


dataset_stat=[]
def create_stat_data():
 
    for img in os.listdir(datadir):  
      try:
        img_array=cv2.imread(os.path.join(datadir, img))#, cv2.IMREAD_GRAYSCALE)
        new_array=cv2.resize(img_array, (128, 128))  
        new_array1=new_array/255 
        new_array2=transform(new_array1)
        dataset_stat.append(new_array2)
      except Exception as e:
        print(e)

create_stat_data()
arr_stack_stack=torch.stack(dataset_stat)


stat_train_loader=DataLoader(arr_stack_stack, batch_size=32, shuffle=False)

def get_mean_and_std(loader):
  mean=0.
  std=0.
  total_images_count=0
  for images in loader:
    image_count_in_a_bach=images.size(0)
    images=images.view(image_count_in_a_bach, images.size(1), -1)
    mean +=images.mean(2).sum(0)
    std +=images.std(2).sum(0)
    total_images_count +=image_count_in_a_bach

  mean /=total_images_count
  std /=total_images_count

  return mean, std

get_mean_and_std(stat_train_loader)

(tensor([0.6090, 0.6004, 0.5262], dtype=torch.float64),
 tensor([0.2978, 0.2828, 0.3395], dtype=torch.float64))

In [ ]:
###############----TEST DATASET DOLPHIN_HORSE CREATION-----

stats = ((0.6090, 0.6004, 0.5262), (0.2978, 0.2828, 0.3395))
test_tfms3 = transforms.Compose([
                         transforms.ToPILImage(),
                         transforms.Resize((128, 128)),
                         #transforms.RandomHorizontalFlip(),
                         #transforms.RandomCrop(128, padding=4, padding_mode='reflect'),
                         transforms.ToTensor(),
                         transforms.Normalize(*stats,inplace=True)]) 


dir_test="/content/test" 
test_categories=['horse/HorseTest/', 'dolphin/DolphinTest/', 'dolphin_horse/DolphinHorseTest/']


def create_test_data():
  for category in test_categories:
    path_test=os.path.join(dir_test, category)
    #if category=="horse/HorseTest/":
     # class_num_test=torch.tensor([1,0], dtype=torch.float32)
    #if category=="dolphin/DolphinTest/":
     # class_num_test=torch.tensor([0,1], dtype=torch.float32)
    if category=="dolphin_horse/DolphinHorseTest/":
      class_num_test=torch.tensor([1,1], dtype=torch.float32)
      for img in os.listdir(path_test):  
        try:
          img_array=cv2.imread(os.path.join(path_test, img))  
          new_array0=test_tfms3(img_array) 
          #new_array=cv2.resize(img_array, (128, 128))
          #new_array1=new_array0/255 
          #test_np = np.array(new_array1)
          #test_np_float32=test_np.astype('float32')
          #test_tensor0 = torch.from_numpy(test_np_float32)
          #test_tensor=new_array0.reshape(3,128,128)
          test_list.append([new_array0, class_num_test])
        except Exception as e:
          print(e)

create_test_data()

In [ ]:
inputs_test=[]
targets_test0=[]

for image, label in test_list:
  inputs_test.append(image)
  targets_test0.append(label)

stack_test_list=torch.stack(inputs_test)
stack_test_targets=torch.stack(targets_test0)
arr_stack_test_reshape=stack_test_list.reshape(-1,3,128,128) 

batch_size=32


dataset_tensor_test=TensorDataset(arr_stack_test_reshape, stack_test_targets)
test_dl = DataLoader(dataset_tensor_test, batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_dl2=DataLoader(arr_stack_test_reshape, batch_size, shuffle=False, num_workers=2, pin_memory=True)
#test_dl2device=DeviceDataLoader(test_dl, device)





In [ ]:

result_test=evaluate(model2, test_dl)
result_test


{'Check': 1.0, 'val_acc': 0.9947090148925781, 'val_loss': 0.039370790123939514}

In [ ]:
def F_score(output, label, threshold=0.5, beta=1):
    prob = output > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)

    # F2 = 2* precision * recall / (precision + recall)

    return F2.mean(0)
test=torch.from_numpy(test_numpy)
F_score(test, stack_test_targets)

tensor(0.7928)

In [ ]:
print(test_numpy[0][0])

[0.98804444 0.98063207]


In [ ]:
counter=0
for i in range(len(test_list)):
  print(test_list[i][1], test_numpy[i])
  if test_list[i][1][0]==0 and test_numpy[i][0]<0.5 and test_list[i][1][1]==1 and test_numpy[i][1]>=0.5:
    counter+=1
    print(test_list[i][1][0], test_numpy[i][0], test_list[i][1][1], test_numpy[i][1])
  if test_list[i][1][0]==1 and test_numpy[i][0]>=0.5 and test_list[i][1][1]==0 and test_numpy[i][1]<0.5:
    counter+=1
    print(test_list[i][1][0], test_numpy[i][0], test_list[i][1][1], test_numpy[i][1])
  if test_list[i][1][0]==1 and test_numpy[i][0]>=0.5 and test_list[i][1][1]==1 and test_numpy[i][1]>=0.5:
    counter+=1
    print(test_list[i][1][0], test_numpy[i][0], test_list[i][1][1], test_numpy[i][1])

result=counter/len(test_list)  
print(counter)
print(result)


In [ ]:
stack_test_targets

In [ ]:
def F_score(output, label, threshold=0.5, beta=1):
    prob = output > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)

    # F2 = 2* precision * recall / (precision + recall)

    return F2.mean(0)

for batch in test_dl:
  inputs, labels=batch
  a=model2(inputs)
  score = F_score(a, labels)
  print(score)

tensor(0.7074)
tensor(0.8113)
tensor(0.9189)


In [ ]:
a=result_test[0][0][0].detach().numpy()
counterH=0
counterD=0
counterHD=0
for x in range(len(test_result2)):
  for i in range(len(result[x])):
    if result[0][i][0].detach().numpy()>0.5 and result[0][i][1].detach().numpy()<0.5:
      counterH+=1
    if result[0][i][0].detach().numpy()<0.5 and result[0][i][1].detach().numpy()>0.5:
      counterD+=1
    if result[0][i][0].detach().numpy()>0.5 and result[0][i][1].detach().numpy()>0.5:
      counterHD+=1

print("Horse= ", counterH)
print("Dolphin= ", counterD)
print("HoreseDolphin= ", counterHD)
#horse=35 Dolphin=27 HorseDolphin=12

In [ ]:
#Prediction for image containing just dolphin (it should be around [0,1])

img_array=cv2.imread("/content/drive/MyDrive/tesztd.jpg")#, cv2.IMREAD_GRAYSCALE)
#new_array=cv2.resize(img_array, (128, 128))
new_array0=test_tfms3(img_array) 
#new_array1=new_array/255 
#test_np = np.array(new_array1)
#test_np_float32=test_np.astype('float32')
#test_tensor0 = torch.from_numpy(test_np_float32)
test_tensor=new_array0.reshape(3,128,128)

predict_image(test_tensor, model2)

tensor([[0.0010, 0.9997]], grad_fn=<SigmoidBackward>)

In [ ]:
image2device=to_device(test_tensor.unsqueeze(0), device)
model(image2device)

tensor([[0., 1.]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [ ]:
#Prediction for image containing just horse (it should be around [1,0]) 

img_array=cv2.imread("/content/drive/MyDrive/teszth2.jpg")#, cv2.IMREAD_GRAYSCALE)
new_array=cv2.resize(img_array, (128, 128))
new_array1=new_array/255 
test_np = np.array(new_array1)
test_np_float32=test_np.astype('float32')
test_tensor0 = torch.from_numpy(test_np_float32)
test_tensor=test_tensor0.reshape(3,128,128)

predict_image(test_tensor, model2)

tensor([[0.9890, 0.0061]], grad_fn=<SigmoidBackward>)

In [ ]:
#----------------Using model resnet18 PRETRAINED on ImageNet to get predictions, the result is bad------


#No transfer learning
import torchvision.models as models

resnet18 = models.resnet18()
resnet18



In [ ]:
from PIL import Image
from torchvision import transforms
input_image = Image.open("/content/drive/MyDrive/teszth2.jpg")
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)

In [ ]:
def predict_image(img, model):
    xb = img.unsqueeze(0)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    return preds.item()

In [ ]:
!unzip "/content/drive/MyDrive/942d3a0ac09ec9e5eb3a-238f720ff059c1f82f368259d1ca4ffa5dd8f9f5.zip" -d "/content/"

In [ ]:
path="/content/942d3a0ac09ec9e5eb3a-238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt"
with open(path) as f:
  classes = [i.strip() for i in f.readlines()]

classes2=[]
classes2.append(classes[0][1:])
for i in range(1, len(classes)-1):
  classes2.append(classes[i])


classes3=[]
for i in classes2:
  classes3.append(i.split(":"))



In [ ]:
#Prediction for image containing just horse  


prediction=predict_image(input_tensor, resnet18)

for i in classes3:

  if i[0]==str(prediction):
    print(prediction, i[0],i[1])

In [ ]:
#--------------------------TRANSFER LEARNING------------------------------

import torchvision.models as models
from tqdm.notebook import tqdm

def F_score(output, label, threshold=0.5, beta=1):
    prob = output > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2.mean(0)

class MultilabelImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, targets = batch 
        out = self(images)                      
        loss = F.binary_cross_entropy(out, targets)      
        return loss
    
    def validation_step(self, batch):
        images, targets = batch 
        out = self(images)                           # Generate predictions
        loss = F.binary_cross_entropy(out, targets)  # Calculate loss
        score = F_score(out, targets)
        return {'val_loss': loss.detach(), 'val_score': score.detach() }
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_scores = [x['val_score'] for x in outputs]
        epoch_score = torch.stack(batch_scores).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_score': epoch_score.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.4f}, train_loss: {:.4f}, val_loss: {:.4f}, val_score: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_score']))
        


In [ ]:
class HorseDolphin(MultilabelImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet34(pretrained=True)
        # Replace last layer
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Linear(num_ftrs, 2)
    
    def forward(self, xb):
        return torch.sigmoid(self.network(xb))
    
    def freeze(self):
        # To freeze the residual layers
        for param in self.network.parameters():
            param.require_grad = False
        for param in self.network.fc.parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.network.parameters():
            param.require_grad = True



In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

device = get_default_device()
device

train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
model = to_device(HorseDolphin(), device)


In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in tqdm(range(epochs)):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader, leave=False):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
model.freeze()

In [ ]:
history = [evaluate(model, val_dl)]
history


In [ ]:
epochs = 5
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam


history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, 
                         grad_clip=grad_clip, 
                         weight_decay=weight_decay, 
                         opt_func=opt_func)

In [ ]:
model.unfreeze()

In [ ]:

history += fit_one_cycle(epochs, 0.001, model, train_dl, val_dl, 
                         grad_clip=grad_clip, 
                         weight_decay=weight_decay, 
                         opt_func=opt_func)


In [ ]:

scores = [x['val_score'] for x in history]
plt.plot(scores, '-x')
plt.xlabel('epoch')
plt.ylabel('score')
plt.title('F1 score vs. No. of epochs')

In [ ]:

train_losses = [x.get('train_loss') for x in history]
val_losses = [x['val_loss'] for x in history]
plt.plot(train_losses, '-bx')
plt.plot(val_losses, '-rx')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Training', 'Validation'])
plt.title('Loss vs. No. of epochs');

In [ ]:
torch.save(model.state_dict(), 'pics.modelparameters')
model2=HorseDolphin()
model2.load_state_dict(torch.load('pics.modelparameters'))
model2.state_dict

def predict_image(img, model):
    xb = img.unsqueeze(0)
    yb = model(xb)
    #_, preds  = torch.max(yb, dim=1)
    return yb

#Prediction for image containing dolphin and horse (it should be around [1,1])
img_array=cv2.imread("/content/drive/MyDrive/tesztdh.jpg")#, cv2.IMREAD_GRAYSCALE)
new_array=cv2.resize(img_array, (128, 128))
new_array1=new_array/255 
test_np = np.array(new_array1)
test_np_float32=test_np.astype('float32')
test_tensor0 = torch.from_numpy(test_np_float32)
test_tensor=test_tensor0.reshape(3,128,128)

predict_image(test_tensor, model2)

In [ ]:
#Prediction for image containing just dolphin (it should be around [0,1])

img_array=cv2.imread("/content/drive/MyDrive/tesztd2.jpg")#, cv2.IMREAD_GRAYSCALE)
new_array=cv2.resize(img_array, (128, 128))
new_array1=new_array/255 
test_np = np.array(new_array1)
test_np_float32=test_np.astype('float32')
test_tensor0 = torch.from_numpy(test_np_float32)
test_tensor=test_tensor0.reshape(3,128,128)

predict_image(test_tensor, model2)

In [ ]:
#Prediction for image containing just horse (it should be around [1,0]) 

img_array=cv2.imread("/content/drive/MyDrive/teszth2.jpg")#, cv2.IMREAD_GRAYSCALE)
new_array=cv2.resize(img_array, (128, 128))
new_array1=new_array/255 
test_np = np.array(new_array1)
test_np_float32=test_np.astype('float32')
test_tensor0 = torch.from_numpy(test_np_float32)
test_tensor=test_tensor0.reshape(3,128,128)

predict_image(test_tensor, model2)

In [ ]:
torch.save(model.state_dict(), 'pics.modelparameters')
model2=ResNet9(3,2)
model2.load_state_dict(torch.load('pics.modelparameters'))
model2.state_dict

def predict_image(img, model):
    #xb = img.unsqueeze(0)
    yb = model(img)
    #_, preds  = torch.max(yb, dim=1)
    return yb







In [ ]:
path="."
os.chdir(path)
os.makedirs("/content/tesztek")

In [ ]:
Archive('/content/HorseDolphinTest.rar').extractall('/content/tesztek/')



In [ ]:
teszt_tfms11 = transforms.Compose([transforms.Resize((128, 128)),
                         transforms.ToTensor()])

ds_teszt=ImageFolder("/content/tesztek", teszt_tfms11)

ds_teszt0=[]
for i in ds_teszt:
  ds_teszt0.append(i[0])

ds_teszt1=torch.stack(ds_teszt0)

#batch_size=8
#ds_teszt1 = DataLoader(ds_teszt0, batch_size, shuffle=True, num_workers=2, pin_memory=True)

#for batch in ds_teszt1:
 # print(torch.stack(batch))
a=predict_image(ds_teszt1, model2)

counterh=0
counterd=0
counterdh=0
for i in a:
  if i[0]>0.5 and i[1]<0.5:
    counterh +=1
  if i[0]<0.5 and i[1]>0.5:
    counterd +=1
  if i[0]>0.5 and i[1]>0.5: 
    counterdh+=1

print(counterh, counterd, counterdh)
